## Creando nuestro propio Churn Analysis 🤓📊

En este lab aprenderás:

* [Tensorflow](https://www.tensorflow.org/)
* [Keras](https://keras.io/)
* Descargar un dataset, prepararlo, entrenarlo, realizar finetuning y guardarlo.


### 1) Descarga del dataset 🤓

Utilizaremos un conjunto de datos de un proveedor de Telecomunicaciones para su Programa de Retención.
<br>Para más detalle acá se puede ver el dataset de Kaggle: [Telco Customer Churn](https://www.kaggle.com/datasets/blastchar/telco-customer-churn/data).


In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle datasets list -s telco-customer-churn

In [ ]:
!kaggle datasets download -d blastchar/telco-customer-churn

In [ ]:
!unzip '/content/telco-customer-churn.zip'

### 2) Preparación de la data 👌

#### 2.1) Instalamos las dependencias 🙌

In [ ]:
!pip install ydata-profiling

In [ ]:
import joblib
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc

#### 2.2) Explorar Dataset 🔍

In [ ]:
!ls

In [ ]:
df = pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv", sep=",")

**Tip:** Para visualizar todas las columnas del dataframe

In [ ]:
df.head(3)

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df.head(3)

#### 2.3) EDA (Análisis Exploratorio de Datos)

Visualizamos qué tenemos en el dataframe

In [ ]:
# Crear un resumen utilizando funciones nativas de pandas
def summarize_dataframe_with_pandas(df):
    summary = df.describe(include='all').T  # Descripción general
    summary['Type'] = df.dtypes  # Tipos de datos
    summary['Unique Values'] = df.nunique()  # Cantidad de valores únicos
    summary['Examples'] = df.apply(lambda col: col.dropna().unique()[:3])  # Ejemplos de valores

    # Reorganizar columnas para mejor visualización
    summary = summary[['Type', 'Unique Values', 'Examples']]
    return summary

In [ ]:
summarize_dataframe_with_pandas(df)

Una visualización más detallada e interactiva

In [ ]:
from ydata_profiling import ProfileReport

ProfileReport(df, minimal=True)

#### 2.4) Valores únicos

Eliminar columna con valores únicos

In [ ]:
df = df.drop('customerID', axis=1)

Eliminar columna con que puede generar Bias o Sesgo

In [ ]:
df = df.drop('gender', axis=1)

#### 2.5) Valores faltantes

In [ ]:
# Evaluar cantidad de valores faltantes
df.isnull().sum()

#### 2.6) Columnas Categóricas

Reeplazo de valores binarios en columnas categóricas

In [ ]:
# Evitar warning por uso de Replace
pd.set_option('future.no_silent_downcasting', True)

In [ ]:
categorical_columns = list(df.select_dtypes(include='O').keys())

for i in categorical_columns:
    df[i] = df[i].replace('Yes', 1)
    df[i] = df[i].replace('No', 0)

Label Encoder

In [ ]:
# Inicializar y aplicar LabelEncoder único
label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))  # Codificar las columnas categóricas
    label_encoders[col] = le  # Guardar el codificador único para cada columna

In [ ]:
# Guardar los label encoders
joblib.dump(label_encoders, 'label_encoders.pkl')

#### 2.7) Columnas Numéricas

Escalar la data

In [ ]:
scale_cols = ['tenure','MonthlyCharges','TotalCharges']

scale = MinMaxScaler()
df[scale_cols] = scale.fit_transform(df[scale_cols])

In [ ]:
# Guardar el escalado de datos
joblib.dump(scale, 'scaler.pkl')

### 3) Entrenamiento 💪

In [ ]:
x = df.drop('Churn', axis=1)
y = df['Churn']

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2)

print(xtrain.shape, ytrain.shape)

In [ ]:
print(xtest.shape, ytest.shape)

### 4) Red Neuronal 😨

In [ ]:
!pip install keras-tuner

In [ ]:
from keras_tuner import RandomSearch

In [ ]:
# Obtener el número de columnas de entrenamiento
num_columns = 18

**Función para construir el modelo**

In [ ]:
def build_model(hp):
    model = keras.Sequential([
        keras.layers.Input(shape=(num_columns,))
        ])

    # Primera capa con ajuste de unidades y función de activación
    model.add(keras.layers.Dense(units=hp.Int('units_layer1', min_value=10, max_value=16, step=2),
                    activation=hp.Choice('activation_layer1', values=['relu', 'tanh'])))

    # Dropout ajustable
    model.add(keras.layers.Dropout(rate=hp.Float('dropout_layer1', min_value=0.0, max_value=0.5, step=0.1)))

    # Segunda capa opcional
    if hp.Boolean('second_layer'):
        model.add(keras.layers.Dense(units=hp.Int('units_layer2', min_value=5, max_value=10, step=1),
                        activation=hp.Choice('activation_layer2', values=['relu', 'tanh'])))
        model.add(keras.layers.Dropout(rate=hp.Float('dropout_layer2', min_value=0.0, max_value=0.5, step=0.1)))

    # Capa de salida
    model.add(keras.layers.Dense(1, activation='sigmoid'))

    # Compilación
    model.compile(
        optimizer=hp.Choice('optimizer', values=['adam', 'adamW']),
        loss='binary_crossentropy',
        metrics=['accuracy'])

    return model

**Configurar KerasTuner**

In [ ]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',  # Métrica a optimizar
    max_trials=10,             # Número de combinaciones a probar
    executions_per_trial=2,    # Número de ejecuciones por combinación
    directory='my_dir',        # Carpeta para guardar resultados
    project_name='churn_tuning'  # Nombre del proyecto
)

**Ejecutar la búsqueda**

Vamos a probar con 20 epochs

In [ ]:
tuner.search(xtrain, ytrain, epochs=20, validation_data=(xtest, ytest))

**Obtener el mejor modelo**

In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Mejores hiperparámetros encontrados:")
print(f"- Unidades primera capa: {best_hps.get('units_layer1')}")
print(f"- Optimizer: {best_hps.get('optimizer')}")
print(f"- Dropout primera capa: {best_hps.get('dropout_layer1')}")

**Construir el mejor modelo**

In [ ]:
best_model = tuner.hypermodel.build(best_hps)

In [ ]:
keras.utils.plot_model(best_model, show_shapes=True)

**Entrenar el mejor modelo**

Probamos con 50 epochs

In [ ]:
# Agregamos el Early Stopping
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
history = best_model.fit(xtrain, ytrain, epochs=50, validation_data=(xtest, ytest), callbacks=[early_stop])

### 5) Métricas 📊

***Ahora las métricas obtenidas son para best_model.***

**Matriz de Confusión**

In [ ]:
y_pred = (best_model.predict(xtest) > 0.5)  # Convertir las probabilidades a 0 o 1
cm = confusion_matrix(ytest, y_pred)

plt.figure(figsize=(6, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Greens', xticklabels=['No', 'Yes'], yticklabels=['No', 'Yes'])
plt.title("Matriz de Confusión")
plt.xlabel("Predicción")
plt.ylabel("Real")
plt.show()

**Reporte de Métricas**

In [ ]:
report = classification_report(ytest, y_pred, target_names=['No', 'Yes'])
print(report)

**Gráfica de Accuracy**

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Accuracy Entrenamiento')
plt.plot(history.history['val_accuracy'], label='Accuracy Validación')
plt.title('Accuracy vs Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

**Gráfica de Loss**

In [ ]:
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Loss Entrenamiento')
plt.plot(history.history['val_loss'], label='Loss Validación')
plt.title('Loss vs Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

**Curva ROC**

In [ ]:
# Obtener las predicciones de probabilidades
y_pred_train = (best_model.predict(xtrain) > 0.5)  # Convertir las probabilidades a 0 o 1

# Calcular las métricas de la curva ROC para entrenamiento
fpr_train, tpr_train, _ = roc_curve(ytrain, y_pred_train)
roc_auc_train = auc(fpr_train, tpr_train)

# Calcular las métricas de la curva ROC para validación
fpr_val, tpr_val, _ = roc_curve(ytest, y_pred)
roc_auc_val = auc(fpr_val, tpr_val)

# Graficar ambas curvas ROC
plt.figure(figsize=(8, 6))
plt.plot(fpr_train, tpr_train, color='blue', lw=2, label=f'Train ROC (AUC = {roc_auc_train:.2f})')
plt.plot(fpr_val, tpr_val, color='green', lw=2, label=f'Validation ROC (AUC = {roc_auc_val:.2f})')
plt.plot([0, 1], [0, 1], color='red', linestyle='--', lw=2, label='Random guess')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=14)
plt.ylabel('True Positive Rate', fontsize=14)
plt.title('Receiver Operating Characteristic (ROC)', fontsize=16)
plt.legend(loc="lower right", fontsize=12)
plt.grid(alpha=0.3)
plt.show()

### 6) Guardar el modelo 💾

In [ ]:
# Guardar en formato HDF5
best_model.save('mi_modelo_entrenado.keras')



**El modelo pesa ~28 KB.**

### 7) Hacer Predicciones en Producción 🤙

In [ ]:
import joblib
import pandas as pd

from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

Cargar el modelo una vez (al inicio de la aplicación)

In [ ]:
# Cargar los objetos de preprocesamiento guardados
label_encoders = joblib.load('label_encoders.pkl')
scaler = joblib.load('scaler.pkl')

# Cargar el modelo entrenado
model = load_model('mi_modelo_entrenado.keras', compile=False)

Función para predicción / inferencia

In [ ]:
# Función para preprocesar datos con los objetos guardados
def preprocess_data(df):
    # Eliminar columnas irrelevantes
    df = df.drop(['customerID', 'gender'], axis=1, errors='ignore')

    # Reemplazar valores "Yes"/"No" por 1/0
    categorical_columns = list(df.select_dtypes(include='O').keys())
    for i in categorical_columns:
        df[i] = df[i].replace('Yes', 1)
        df[i] = df[i].replace('No', 0)

    # Aplicar Label Encoding usando los objetos guardados
    for col, le in label_encoders.items():
        if col in df.columns:
            # Asegurar que los valores sean cadenas antes de la transformación
            df[col] = df[col].astype(str)
            try:
                df[col] = le.transform(df[col])
            except ValueError as e:
                raise ValueError(
                    f"Error al transformar la columna '{col}'. "
                    f"Asegúrate de que los valores en los nuevos datos coincidan con los datos de entrenamiento. "
                    f"Detalles: {e}"
                )

    # Aplicar escalado usando el scaler guardado
    scale_cols = ['tenure', 'MonthlyCharges', 'TotalCharges']
    df[scale_cols] = scaler.transform(df[scale_cols])

    return df

In [ ]:
def make_prediction(single_case):
  # Convertir el caso único en DataFrame
  single_case_df = pd.DataFrame(single_case)

  # Preprocesar el caso único
  processed_single_case = preprocess_data(single_case_df)

  # Separar las características del target si aplica
  X_single_case = processed_single_case.drop('Churn', axis=1, errors='ignore')

  # Realizar predicción
  prediction = model.predict(X_single_case)

  score = prediction[0][0] * 100
  print(f"Probabilidad que abandone: {score:.2f} %")

  prediction_binary = (prediction > 0.5).astype(int)
  print("Churn: ", prediction_binary[0][0])

Ejemplos de uso

In [ ]:
# Churn: Yes / 1
single_case = {
    "customerID": ["9237-HQITU"],
    "gender": ["Female"],
    "SeniorCitizen": [0],
    "Partner": ["No"],
    "Dependents": ["No"],
    "tenure": [2],
    "PhoneService": ["Yes"],
    "MultipleLines": ["No"],
    "InternetService": ["Fiber optic"],
    "OnlineSecurity": ["No"],
    "OnlineBackup": ["No"],
    "DeviceProtection": ["No"],
    "TechSupport": ["No"],
    "StreamingTV": ["No"],
    "StreamingMovies": ["No"],
    "Contract": ["Month-to-month"],
    "PaperlessBilling": ["Yes"],
    "PaymentMethod": ["Electronic check"],
    "MonthlyCharges": [70.70],
    "TotalCharges": [151.65],
}

make_prediction(single_case)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
Probabilidad que abandone: 63.90 %
Churn:  1


In [ ]:
# Churn: No / 0
single_case = {
    "customerID": ["7795-CFOCW"],
    "gender": ["Male"],
    "SeniorCitizen": [0],
    "Partner": ["No"],
    "Dependents": ["No"],
    "tenure": [45],
    "PhoneService": ["No"],
    "MultipleLines": ["No phone service"],
    "InternetService": ["DSL"],
    "OnlineSecurity": ["Yes"],
    "OnlineBackup": ["No"],
    "DeviceProtection": ["Yes"],
    "TechSupport": ["Yes"],
    "StreamingTV": ["No"],
    "StreamingMovies": ["No"],
    "Contract": ["One year"],
    "PaperlessBilling": ["No"],
    "PaymentMethod": ["Bank transfer (automatic)"],
    "MonthlyCharges": [42.30],
    "TotalCharges": [1840.75],
}

make_prediction(single_case)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Probabilidad que abandone: 4.02 %
Churn:  0


In [ ]:
# Churn: No / 0
single_case = {
    "customerID": ["7590-VHVEG"],
    "gender": ["Female"],
    "SeniorCitizen": [0],
    "Partner": ["Yes"],
    "Dependents": ["No"],
    "tenure": [1],
    "PhoneService": ["No"],
    "MultipleLines": ["No phone service"],
    "InternetService": ["DSL"],
    "OnlineSecurity": ["No"],
    "OnlineBackup": ["Yes"],
    "DeviceProtection": ["No"],
    "TechSupport": ["No"],
    "StreamingTV": ["No"],
    "StreamingMovies": ["No"],
    "Contract": ["Month-to-month"],
    "PaperlessBilling": ["Yes"],
    "PaymentMethod": ["Electronic check"],
    "MonthlyCharges": [29.85],
    "TotalCharges": [29.85],
}

make_prediction(single_case)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Probabilidad que abandone: 49.35 %
Churn:  0


In [ ]:
# Churn: Yes / 1
single_case = {
    "customerID": ["3668-QPYBK"],
    "gender": ["Male"],
    "SeniorCitizen": [0],
    "Partner": ["No"],
    "Dependents": ["No"],
    "tenure": [2],
    "PhoneService": ["Yes"],
    "MultipleLines": ["No"],
    "InternetService": ["DSL"],
    "OnlineSecurity": ["Yes"],
    "OnlineBackup": ["Yes"],
    "DeviceProtection": ["No"],
    "TechSupport": ["No"],
    "StreamingTV": ["No"],
    "StreamingMovies": ["No"],
    "Contract": ["Month-to-month"],
    "PaperlessBilling": ["Yes"],
    "PaymentMethod": ["Mailed check"],
    "MonthlyCharges": [53.85],
    "TotalCharges": [108.15],
}

make_prediction(single_case)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Probabilidad que abandone: 26.36 %
Churn:  0


### 8) Conclusiones

- Aprender sobre los distintos objetos y métodos que nos ofrece Tensorflow + Keras.

- Realizar el proceso completo de entrenamiento de un modelo con Tensorflow + Keras.

- Aprender tips sobre implementación con el uso de la GPU.

<br>
<br>
<br>

---

<br>
<br>


<img src="https://static.platzi.com/media/avatars/platziteam_8cfe6fc7-1246-4c9a-9f5d-d10d467443ee.png" width="100px">

[Platzi](https://platzi.com/) 🚀

